In [3]:
import os
import zipfile
import xml.etree.ElementTree as ET

In [4]:
current_directory = os.getcwd()

print(f"현재 작업 경로: {current_directory}")

현재 작업 경로: c:\Users\PC\OneDrive\바탕 화면\git_repo\ML-DL-Training\AAS


In [5]:
def read_aasx_file(aasx_path):
    """ AASX 파일을 읽기 """
    with zipfile.ZipFile(aasx_path, 'r') as aasx:
        # ZIP 내부의 파일 목록 출력 (AASX 내부 구조 확인)
        file_list = aasx.namelist()
        print(" - AASX 파일 내의 파일 목록:")
        for file_name in file_list:
            print(file_name)
        
        # AAS 데이터가 저장되어 있는 XML 파일 선택
        target_file = 'aasx/Injection_Molding_Machine/Injection_Molding_Machine.aas.xml'  # 현재 저장되어 있는 사출기 파일 기준으로 작성함
        
        if target_file in file_list:
            # XML 파일을 읽어서 파싱
            with aasx.open(target_file) as xml_file:
                tree = ET.parse(xml_file)
                root = tree.getroot()

                # 네임스페이스 제거
                remove_namespace(root)
                return root
        else:
            print(f"{target_file} 파일을 찾을 수 없습니다.")
            return None

def remove_namespace(xml_data):
    """ 네임스페이스 제거 """
    for elem in xml_data.iter():
        if '}' in elem.tag:
            elem.tag = elem.tag.split('}', 1)[1]  # 네임스페이스를 제거한 태그명으로 재설정
    return xml_data

def extract_submodel_data(element, processed_submodels, level=0):
    """ Submodel 정보와 하위 요소 추출 """
    indent = "  " * level  # 계층적 출력을 위한 들여쓰기

    # 태그 타입과 idShort 추출
    element_id_short = element.find("idShort")
    
    # 요소의 유형을 명시하고 idShort를 출력
    if element_id_short is not None:
        id_short_value = element_id_short.text

        # 이미 처리된 Submodel인지 확인
        if id_short_value not in processed_submodels:
            element_type = element.tag
            print(f"{indent}{element_type}: {id_short_value}")
            processed_submodels.add(id_short_value)  # 중복 방지를 위해 idShort 저장

            # semanticId의 value 값 추출
            semantic_id = element.find("semanticId")
            if semantic_id is not None:
                key_value = semantic_id.find(".//value")
                if key_value is not None:
                    print(f"{indent}SemanticId Value: {key_value.text}")
            
            # submodelElementCollection 처리: value 태그 하위 탐색
            value_elements = element.find("value")
            if value_elements is not None:
                for value_element in value_elements:
                    # 재귀적으로 하위 요소 탐색
                    extract_submodel_data(value_element, processed_submodels, level + 1)

            # submodelElements가 있을 경우 처리
            submodel_elements = element.find("submodelElements")
            if submodel_elements is not None:
                for sub_element in submodel_elements:
                    extract_submodel_data(sub_element, processed_submodels, level + 1)

def find_top_level_submodels(root):
    """ 최상위 Submodel만 찾기 """
    submodels = root.findall(".//submodel")
    return submodels

In [9]:
# 예시 AASX 파일 경로("현재작업디렉토리/aasx_file/파일이름" 형식으로 구성하였습니다)
aasx_file_path = os.path.join(os.getcwd(), 'aasx_file', 'Injection_Molding_Machine.aasx')

# AASX 파일에서 XML 데이터를 읽어옴
aasx_root = read_aasx_file(aasx_file_path)

""" 해당 파일에서 "aasx/aas이름/aas이름.aas.xml"파일이 데이터가 저장되어 있는 xml파일 입니다"""

 - AASX 파일 내의 파일 목록:
aasx/aasx-origin
aasx/Injection_Molding_Machine/Injection_Molding_Machine.aas.xml
_rels/.rels
aasx/_rels/aasx-origin.rels
aasx/Injection_Molding_Machine/_rels/Injection_Molding_Machine.aas.xml.rels
aasx/files/2023-10-05_A5_de2f52d5.pdf
[Content_Types].xml
Injection_Molding_Machine.png
aasx/Quality_P.png


' 해당 파일에서 "aasx/aas이름/aas이름.aas.xml"파일이 데이터가 저장되어 있는 xml파일 입니다'

In [76]:
# Submodel 별 요소 출력
if aasx_root is not None:
    # 이미 처리한 submodel idShort를 저장할 집합(set)
    processed_submodels = set()

    # 최상위 Submodel 정보 추출 (중복을 방지)
    top_level_submodels = find_top_level_submodels(aasx_root)
    
    if top_level_submodels:
        for submodel in top_level_submodels:
            submodel_id_short = submodel.find("idShort")
            # 중복된 Submodel에 대한 전체 출력 건너뜀
            if submodel_id_short is not None and submodel_id_short.text not in processed_submodels:
                print("\nSubmodel 정보:")
                extract_submodel_data(submodel, processed_submodels)
    else:
        print("Submodel 정보를 찾을 수 없습니다.")
else:
    print("XML 데이터를 불러오지 못했습니다.")


Submodel 정보:
submodel: Identification
SemanticId Value: https://www.hsu-hh.de/aut/aas/identification
  property: ManufacturerName
  SemanticId Value: 0173-1#02-AAO677#002
  property: ManufacturerId01
  SemanticId Value: 0173-1#02-AAY812#001
  property: ManufacturerIdProvider
  SemanticId Value: 0173-1#02-AAP796#004
  property: ManufacturerTypId
  SemanticId Value: 0173-1#02-AAO676#003
  property: ManufacturerTypName
  SemanticId Value: 0173-1#02-AAW338#001
  property: ManufacturerTypDescription
  SemanticId Value: 0173-1#02-AAU734#001
  property: SupplierName
  SemanticId Value: 0173-1#02-AAO735#003
  property: SupplierId
  SemanticId Value: 0173-1#02-AAY813#001
  property: SupplierIdProvider
  SemanticId Value: www.company.com/ids/cd/7183_0272_7091_7264
  property: SupplierTypId
  SemanticId Value: 0173-1#02-AAO736#004
  property: SupplierTypName
  SemanticId Value: 0173-1#02-AAM551#002
  property: SupplierTypDescription
  SemanticId Value: 0173-1#02-AAU730#001
  property: TypClass
 